In the dir `amazon_reviews_classification_without_DL` I explore different methods to classify text (in particular amazon reviews) that do not involve Deep Learning. 

Here I will continue in my no-DL journey to see if the techniques introduced in ["EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks"](https://arxiv.org/pdf/1901.11196.pdf) by Jason Wei and Kai Zou help improve the success metrics. 

Bear in mind that these techniques are more suited for DL tasks and small dataset, since they slightly alter the sequence by randomly adding/removing words, replacing synonym or randomly swapping words. For example, with processing methods that do not care about word-order (e.g. topic modeling, tf-idf) randomly swapping words adds no value. Also, in datasets that are large enough EDA might not add much value either. 

Nonetheless, I think is a experiment worth doing, so let me quote the original paper to summarise what EDA is: 

1. **Synonym Replacement (SR)**: Randomly choose n words from the sentence that are not stop words. Replace each of these words with one of its synonyms chosen at random.
2. **Random Insertion (RI)**: Find a random synonym of a random word in the sentence that is not a stop word. Insert that synonym into a random position in the sentence. Do this n times.
3. **Random Swap (RS)**: Randomly choose two words in the sentence and swap their positions. Do this n times.
4. **Random Deletion (RD)**: Randomly remove each word in the sentence with probability p.

In their paper they have a FAQ section that is very useful and I recommend reading (as well as the rest of the paper, of course) 

As well as the use of EDA, there is a second new element I am going to introduce here, and that is the use of [fastai](https://docs.fast.ai/)'s [Tokenizer](https://docs.fast.ai/text.transform.html#Tokenizer). I guess by now you (the reader) are familiar with the fastai library, if not, have a look...is a must. In particular, their tokenizer is, in my opinion, the "superior" solution in the market. Relies on Spacy's tokenizer, and to that adds a number of very convenient utilities that eventually add useful information to the tokenized text. 

Moving on, in this notebooks I will start describing the process from the script `augment.py` in advance. In other words, I assume you have downloaded the data and you have run (or had a look) to the code in `prepare_data.py`. The code there simply: 

1. groups reviews scores into 0 (originally 1 and 2), 1 (originally 3), 2 (originally 4) and 4 (originally 5) and 

2. splits the data into train, validation and test. 

Next stage is augment using EDA. To that aim I modified a bit the code from the [original release](https://github.com/jasonwei20/eda_nlp). Simply, I have wrap it up in a class that you call like this: 

```python
from eda import EDA
```

Have a look to `eda.py` within the `eda` dir. Other than that, let's see how the augmentation works

In [3]:
import pandas as pd
import argparse

from pathlib import Path
# I have temporarily copied the eda module into the notebooks dir so I can run this line    
from eda import EDA

In [4]:
DATA_PATH   = Path('../data/')
# of course, you only augment training
TRAIN_PATH = DATA_PATH/'train'

train = pd.read_csv(TRAIN_PATH/'train.csv')
train = list(train.itertuples(index=False, name=None))

In [5]:
train[0]

('Maybe my feet have grown (they say they continue to grow through out your life).  However, I went up a size from the sneakers I wear and still find these a bit snug, especially by the end of the day when my feet are tired.  Unfortunately, I think another half size would make them too big.',
 2)

the EDA class has two methods, the hidden method `_augemnt`, that I could have also called `augment_sentece` and the method `augment` that I could have also called `augment_docs`. 

In [11]:
eda = EDA(alpha_sr=0.2, num_aug=1, rs=False)

The parameters and attributes are:

In [15]:
eda.__dict__

{'alpha_sr': 0.2,
 'alpha_ri': 0.1,
 'alpha_rs': 0.1,
 'p_rd': 0.1,
 'num_aug': 1,
 'sr': True,
 'ri': True,
 'rs': False,
 'rd': True}

`alpha_xx` are the percentage of words that will be "transformed" during the augmentation process (e.g. `alpha_sr=0.2` means 20% of the words will be replaced by synonyms). `num_aug` is the number of new generated augmented documents/reviews and `sr`, `ri`, etc are booleans indicating whether the corresponding augmentation strategy will be applied. Let's see how it works

In [12]:
eda._augment(train[0][0])

['maybe my feet have grown (they say they continue to grow through out your life). however, i went up a size from the sneakers i wear and still find these a bit snug, especially by the end of the day when my feet are tired. unfortunately, i think another half size would make them too big.',
 'maybe my invertebrate foot have develop ( they say they remain to mature through out your lifespan ) . however , i went up a sizing from the sneakers i wear and still find these a bit snuggery , particularly by the cease of the mean solar day when my invertebrate foot are tired . alas , i think another one half sizing would make them too prominent .',
 'maybe my feet have grown ( they say they continue to grow through out your life ) . however , i went done up a size from the too sneakers besides i wear and still find these remember a bit snug lifespan , especially by the end of the day when my feet are foundation tired . unfortunately , i think another half size would make them too big .',
 'mayb

And to run it on a number of docs/reviews (let's just augment the first 1000 here for illustration purposes)

In [17]:
augmented_reviews = eda.augment(train[:1000])

100%|██████████| 1000/1000 [00:05<00:00, 187.36it/s]


In [19]:
len(augmented_reviews[0])

4

In [20]:
augmented_reviews[0]

[('maybe my feet have grown (they say they continue to grow through out your life). however, i went up a size from the sneakers i wear and still find these a bit snug, especially by the end of the day when my feet are tired. unfortunately, i think another half size would make them too big.',
  2),
 ('maybe my have grown ( they say they continue to grow through your life ) . however , a size from the sneakers i wear and still find these a bit snug , by end of the day when my feet tired . unfortunately , i think another half size make them too big .',
  2),
 ('perhaps my fundament have produce ( they say they continue to acquire through out your life ) . however , i went up a size of it from the stool pigeon i put on and still find these a bit snug , specially by the end of the day when my fundament are wear upon . alas , i recall another one half size of it would make them too adult .',
  2),
 ('maybe my banal feet have grown ( they say they continue to grow through out your life ) . ho

Onto the next notebook